In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import files
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Please upload the CSV file.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS with brand-specific medians, fallback to dataset-wide median
    def impute_dimensions(group):
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]

        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not all(isinstance(v, (int, float)) for v in x) else x
        )
        return group

    df = df.groupby('BRAND').apply(impute_dimensions, include_groups=False).reset_index()

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (relaxed to 2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in ENTITY_LENGTH or product_size
    original_len = len(df)
    df = df.dropna(subset=['ENTITY_LENGTH', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in ENTITY_LENGTH or product_size. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train):
    print("Training ideal packaging model...")
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Evaluate
    y_pred_train = model.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {mse_train:.2f}, Train R²: {r2_train:.2f}")

    return model

# Step 4: Detect Flaws
def detect_flaws(df_test, ideal_model):
    X_test = df_test[['ENTITY_LENGTH']].values
    df_test['ideal_box_volume'] = ideal_model.predict(X_test)
    df_test['ideal_product_size'] = df_test['ideal_box_volume'] * 0.9

    # Detect flaws
    df_test['design_flaw'] = df_test['product_size'] > 2 * df_test['ideal_product_size']
    df_test['packaging_flaw'] = df_test['box_volume'] > 1.5 * df_test['ideal_box_volume']

    # Excess volume calculation
    df_test['excess_volume'] = df_test.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_test['recommendation'] = df_test.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    return df_test

# Step 5: Main Execution
def main():
    install_dependencies()

    file_path = "/content/amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Train model
    X_train = df_train[['ENTITY_LENGTH']].values
    y_train = df_train['box_volume']
    ideal_model = train_ideal_packaging_model(X_train, y_train)

    # Detect flaws
    df_valid = detect_flaws(df_valid, ideal_model)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_flaw_analysis.csv"
    output_df.to_csv(output_file, index=False)
    print(f"Results saved to '{output_file}'.")
    files.download(output_file)

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()


Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 97 rows
After outlier removal: 74 rows
Dropped 2 rows with NaN in ENTITY_LENGTH or product_size. Remaining: 72
Training set size: 57 records, Validation set size: 15 records
Training ideal packaging model...
Ideal Packaging Model - Train MSE: 878940.35, Train R²: 0.08
Results saved to 'packaging_flaw_analysis.csv'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Detected 1 products with design flaws.
Detected 2 products with packaging flaws.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from google.colab import files
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Please upload the CSV file.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS with brand-specific medians, fallback to dataset-wide median
    def impute_dimensions(group):
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]

        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not all(isinstance(v, (int, float)) for v in x) else x
        )
        return group

    df = df.groupby('BRAND').apply(impute_dimensions, include_groups=False).reset_index()

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (relaxed to 2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in ENTITY_LENGTH or product_size
    original_len = len(df)
    df = df.dropna(subset=['ENTITY_LENGTH', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in ENTITY_LENGTH or product_size. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model with Optional Tuning
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid, diagnose_result):
    print("\nTraining ideal packaging model...")
    poly = None

    if diagnose_result in ["overfitting", "moderate"]:
        # Use Ridge regression with hyperparameter tuning
        print("Applying Ridge regression to address overfitting/moderate performance...")
        ridge = Ridge()
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
        grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        print(f"Best Ridge alpha: {grid_search.best_params_['alpha']}")

    elif diagnose_result == "underfitting":
        # Add polynomial features and use Ridge regression
        print("Applying polynomial features and Ridge regression to address underfitting...")
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train)
        X_valid_poly = poly.transform(X_valid)

        ridge = Ridge()
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
        grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_train_poly, y_train)
        model = grid_search.best_estimator_
        print(f"Best Ridge alpha: {grid_search.best_params_['alpha']}")

        # Update X_train and X_valid for evaluation
        X_train = X_train_poly
        X_valid = X_valid_poly
    else:
        # Use standard Linear Regression
        model = LinearRegression()
        model.fit(X_train, y_train)

    # Evaluate on training set
    y_pred_train = model.predict(X_train)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.2f}, Train R²: {train_r2:.2f}")

    # Evaluate on validation set
    y_pred_valid = model.predict(X_valid)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.2f}, Validation R²: {valid_r2:.2f}")

    return model, train_mse, train_r2, valid_mse, valid_r2, poly

# Step 5: Detect Flaws
def detect_flaws(df_valid, ideal_model, poly=None):
    X_valid = df_valid[['ENTITY_LENGTH']].values
    if poly is not None:
        X_valid = poly.transform(X_valid)  # Apply polynomial transformation if used
    df_valid['ideal_box_volume'] = ideal_model.predict(X_valid)
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws
    df_valid['design_flaw'] = df_valid['product_size'] > 2 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.5 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "/content/amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for initial model
    X_train = df_train[['ENTITY_LENGTH']].values
    y_train = df_train['box_volume']
    X_valid = df_valid[['ENTITY_LENGTH']].values
    y_valid = df_valid['box_volume']

    # Train initial model for diagnostics
    initial_model = LinearRegression()
    initial_model.fit(X_train, y_train)
    y_pred_train = initial_model.predict(X_train)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    y_pred_valid = initial_model.predict(X_valid)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Train final model with tuning if needed
    model, train_mse, train_r2, valid_mse, valid_r2, poly = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid, diagnose_result)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, poly)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_flaw_analysis.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")
    files.download(output_file)

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 6 rows with NaN in ENTITY_LENGTH or product_size. Remaining: 107
Training set size: 85 records, Validation set size: 22 records

Model Performance Diagnosis:
Training R²: 0.06, Validation R²: 0.24
Training MSE: 881170.88, Validation MSE: 301726.62
Underfitting detected: Poor performance on both training and validation sets.

Training ideal packaging model...
Applying polynomial features and Ridge regression to address underfitting...
Best Ridge alpha: 100
Ideal Packaging Model - Train MSE: 858774.51, Train R²: 0.09
Ideal Packaging Model - Validation MSE: 351669.98, Validation R²: 0.11

Results saved to 'packaging_flaw_analysis.csv'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Detected 0 products with design flaws.
Detected 2 products with packaging flaws.


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS and ENTITY_LENGTH with brand-specific medians
    def impute_missing(group):
        # Impute dimensions
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]
        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not (isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x)) else x
        )

        # Impute ENTITY_LENGTH
        valid_lengths = group['ENTITY_LENGTH'].apply(
            lambda x: x if isinstance(x, (int, float)) and not np.isnan(x) else None
        )
        valid_lengths_list = [x for x in valid_lengths if x is not None]
        median_length = np.median(valid_lengths_list) if valid_lengths_list else df['ENTITY_LENGTH'].median()

        group['ENTITY_LENGTH'] = group['ENTITY_LENGTH'].fillna(median_length)
        return group

    df = df.groupby('BRAND').apply(impute_missing, include_groups=False).reset_index()

    # Extract individual dimensions
    df['dim_length'] = df['dimensions'].apply(lambda x: x[0])
    df['dim_width'] = df['dimensions'].apply(lambda x: x[1])
    df['dim_height'] = df['dimensions'].apply(lambda x: x[2])

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in key features
    original_len = len(df)
    df = df.dropna(subset=['ENTITY_LENGTH', 'PRICE', 'dim_length', 'dim_width', 'dim_height', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in key features. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid):
    print("\nTraining ideal packaging model with Random Forest...")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Define Random Forest model with hyperparameter tuning
    rf = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5]
    }
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    model = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")

    # Evaluate on training set
    y_pred_train = model.predict(X_train_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}")

    # Evaluate on validation set
    y_pred_valid = model.predict(X_valid_scaled)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.4f}, Validation R²: {valid_r2:.4f}")

    return model, scaler, train_mse, train_r2, valid_mse, valid_r2

# Step 5: Detect Flaws
def detect_flaws(df_valid, model, scaler):
    X_valid = df_valid[['ENTITY_LENGTH', 'PRICE', 'dim_length', 'dim_width', 'dim_height']].values
    X_valid_scaled = scaler.transform(X_valid)
    df_valid['ideal_box_volume'] = model.predict(X_valid_scaled)
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws
    df_valid['design_flaw'] = df_valid['product_size'] > 2 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.5 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for model
    features = ['ENTITY_LENGTH', 'PRICE', 'dim_length', 'dim_width', 'dim_height']
    X_train = df_train[features].values
    y_train = df_train['box_volume']
    X_valid = df_valid[features].values
    y_valid = df_valid['box_volume']

    # Train model
    model, scaler, train_mse, train_r2, valid_mse, valid_r2 = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, scaler)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_flaw_analysis.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 2 rows with NaN in key features. Remaining: 111
Training set size: 88 records, Validation set size: 23 records

Training ideal packaging model with Random Forest...
Best parameters: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Ideal Packaging Model - Train MSE: 33509.4305, Train R²: 0.9661
Ideal Packaging Model - Validation MSE: 238019.7336, Validation R²: 0.3518

Model Performance Diagnosis:
Training R²: 0.97, Validation R²: 0.35
Training MSE: 33509.43, Validation MSE: 238019.73
Overfitting detected: High training performance, but poor validation performance.

Results saved to 'packaging_flaw_analysis.csv'.
Detected 0 products with design flaws.
Detected 1 products with packaging flaws.


In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS and ENTITY_LENGTH with brand-specific medians
    def impute_missing(group):
        # Impute dimensions
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]
        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not (isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x)) else x
        )

        # Impute ENTITY_LENGTH
        valid_lengths = group['ENTITY_LENGTH'].apply(
            lambda x: x if isinstance(x, (int, float)) and not np.isnan(x) else None
        )
        valid_lengths_list = [x for x in valid_lengths if x is not None]
        median_length = np.median(valid_lengths_list) if valid_lengths_list else df['ENTITY_LENGTH'].median()

        group['ENTITY_LENGTH'] = group['ENTITY_LENGTH'].fillna(median_length)
        return group

    df = df.groupby('BRAND').apply(impute_missing, include_groups=False).reset_index()

    # Extract individual dimensions
    df['dim_length'] = df['dimensions'].apply(lambda x: x[0])
    df['dim_width'] = df['dimensions'].apply(lambda x: x[1])
    df['dim_height'] = df['dimensions'].apply(lambda x: x[2])

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in key features
    original_len = len(df)
    df = df.dropna(subset=['ENTITY_LENGTH', 'PRICE', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in key features. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid):
    print("\nTraining ideal packaging model with Random Forest...")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Define Random Forest model with hyperparameter tuning
    rf = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5],
        'min_samples_split': [5, 10],
        'min_samples_leaf': [2, 4]
    }
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    model = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")

    # Evaluate on training set
    y_pred_train = model.predict(X_train_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}")

    # Evaluate on validation set
    y_pred_valid = model.predict(X_valid_scaled)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.4f}, Validation R²: {valid_r2:.4f}")

    # Feature importance
    feature_names = ['ENTITY_LENGTH', 'PRICE']
    importances = model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"Feature: {name}, Importance: {importance:.4f}")

    return model, scaler, train_mse, train_r2, valid_mse, valid_r2

# Step 5: Detect Flaws
def detect_flaws(df_valid, model, scaler):
    X_valid = df_valid[['ENTITY_LENGTH', 'PRICE']].values
    X_valid_scaled = scaler.transform(X_valid)
    df_valid['ideal_box_volume'] = model.predict(X_valid_scaled)
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws
    df_valid['design_flaw'] = df_valid['product_size'] > 2 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.5 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.7, test_size=0.3, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for model
    features = ['ENTITY_LENGTH', 'PRICE']
    X_train = df_train[features].values
    y_train = df_train['box_volume']
    X_valid = df_valid[features].values
    y_valid = df_valid['box_volume']

    # Train model
    model, scaler, train_mse, train_r2, valid_mse, valid_r2 = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, scaler)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_output.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 2 rows with NaN in key features. Remaining: 111
Training set size: 77 records, Validation set size: 34 records

Training ideal packaging model with Random Forest...
Best parameters: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}
Ideal Packaging Model - Train MSE: 709824.5782, Train R²: 0.3219
Ideal Packaging Model - Validation MSE: 448977.2486, Validation R²: -0.0025
Feature: ENTITY_LENGTH, Importance: 0.2550
Feature: PRICE, Importance: 0.7450

Model Performance Diagnosis:
Training R²: 0.32, Validation R²: -0.00
Training MSE: 709824.58, Validation MSE: 448977.25
Underfitting detected: Poor performance on both training and validation sets.

Results saved to 'packaging_output.csv'.
Detected 0 products with design flaws.
Detected 2 products with packaging flaws.


In [5]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn xgboost")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS and ENTITY_LENGTH with brand-specific medians
    def impute_missing(group):
        # Impute dimensions
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]
        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not (isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x)) else x
        )

        # Impute ENTITY_LENGTH
        valid_lengths = group['ENTITY_LENGTH'].apply(
            lambda x: x if isinstance(x, (int, float)) and not np.isnan(x) else None
        )
        valid_lengths_list = [x for x in valid_lengths if x is not None]
        median_length = np.median(valid_lengths_list) if valid_lengths_list else df['ENTITY_LENGTH'].median()

        group['ENTITY_LENGTH'] = group['ENTITY_LENGTH'].fillna(median_length)
        return group

    df = df.groupby('BRAND').apply(impute_missing, include_groups=False).reset_index()

    # Extract individual dimensions
    df['dim_length'] = df['dimensions'].apply(lambda x: x[0])
    df['dim_width'] = df['dimensions'].apply(lambda x: x[1])
    df['dim_height'] = df['dimensions'].apply(lambda x: x[2])

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in key features
    original_len = len(df)
    df = df.dropna(subset=['ENTITY_LENGTH', 'PRICE', 'dim_length', 'dim_width', 'dim_height', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in key features. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid):
    print("\nTraining ideal packaging model with XGBoost...")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Define XGBoost model with hyperparameter tuning
    xgb = XGBRegressor(random_state=42)
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0]
    }
    grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    model = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")

    # Evaluate on training set
    y_pred_train = model.predict(X_train_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}")

    # Evaluate on validation set
    y_pred_valid = model.predict(X_valid_scaled)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.4f}, Validation R²: {valid_r2:.4f}")

    # Feature importance
    feature_names = ['ENTITY_LENGTH', 'PRICE', 'dim_length', 'dim_width', 'dim_height']
    importances = model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"Feature: {name}, Importance: {importance:.4f}")

    return model, scaler, train_mse, train_r2, valid_mse, valid_r2

# Step 5: Detect Flaws
def detect_flaws(df_valid, model, scaler):
    X_valid = df_valid[['ENTITY_LENGTH', 'PRICE', 'dim_length', 'dim_width', 'dim_height']].values
    X_valid_scaled = scaler.transform(X_valid)
    df_valid['ideal_box_volume'] = model.predict(X_valid_scaled)
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws
    df_valid['design_flaw'] = df_valid['product_size'] > 2 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.5 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    # Outlier analysis
    print("\nValidation Set Outlier Analysis:")
    Q1 = df_valid['box_volume'].quantile(0.25)
    Q3 = df_valid['box_volume'].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_valid[df_valid['box_volume'] > Q3 + 1.5 * IQR]
    if not outliers.empty:
        print(f"Found {len(outliers)} potential outliers in validation set 'box_volume':")
        print(outliers[['ENTITY_NAME', 'box_volume', 'product_size']].to_string(index=False))
    else:
        print("No outliers detected in validation set 'box_volume'.")

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.7, test_size=0.3, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for model
    features = ['ENTITY_LENGTH', 'PRICE', 'dim_length', 'dim_width', 'dim_height']
    X_train = df_train[features].values
    y_train = df_train['box_volume']
    X_valid = df_valid[features].values
    y_valid = df_valid['box_volume']

    # Train model
    model, scaler, train_mse, train_r2, valid_mse, valid_r2 = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, scaler)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_output_v3.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 2 rows with NaN in key features. Remaining: 111
Training set size: 77 records, Validation set size: 34 records

Training ideal packaging model with XGBoost...
Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Ideal Packaging Model - Train MSE: 633.6145, Train R²: 0.9994
Ideal Packaging Model - Validation MSE: 53845.8873, Validation R²: 0.8798
Feature: ENTITY_LENGTH, Importance: 0.0351
Feature: PRICE, Importance: 0.0342
Feature: dim_length, Importance: 0.1222
Feature: dim_width, Importance: 0.7074
Feature: dim_height, Importance: 0.1012

Model Performance Diagnosis:
Training R²: 1.00, Validation R²: 0.88
Training MSE: 633.61, Validation MSE: 53845.89
Overfitting detected: High training performance, but poor validation performance.

Validation Set Outlier Analysis:
No outliers detected in validation set

In [7]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn xgboost")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS with brand-specific medians
    def impute_missing(group):
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]
        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not (isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x)) else x
        )
        return group

    df = df.groupby('BRAND').apply(impute_missing, include_groups=False).reset_index()

    # Extract individual dimensions
    df['dim_length'] = df['dimensions'].apply(lambda x: x[0])
    df['dim_width'] = df['dimensions'].apply(lambda x: x[1])
    df['dim_height'] = df['dimensions'].apply(lambda x: x[2])

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in key features
    original_len = len(df)
    df = df.dropna(subset=['dim_length', 'dim_width', 'dim_height', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in key features. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid):
    print("\nTraining ideal packaging model with XGBoost...")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Define XGBoost model for GridSearchCV (without early stopping)
    xgb = XGBRegressor(random_state=42)
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.01, 0.05],
        'subsample': [0.7, 0.9],
        'reg_lambda': [1, 10],
        'reg_alpha': [0, 1]
    }
    grid_search = GridSearchCV(
        xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
    )
    grid_search.fit(X_train_scaled, y_train)

    # Get best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters from GridSearchCV: {best_params}")

    # Train final model with early stopping
    final_model = XGBRegressor(
        random_state=42,
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        reg_lambda=best_params['reg_lambda'],
        reg_alpha=best_params['reg_alpha'],
        early_stopping_rounds=10
    )
    final_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_valid_scaled, y_valid)],
        verbose=True
    )

    # Evaluate on training set
    y_pred_train = final_model.predict(X_train_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}")

    # Evaluate on validation set
    y_pred_valid = final_model.predict(X_valid_scaled)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.4f}, Validation R²: {valid_r2:.4f}")

    # Feature importance
    feature_names = ['dim_length', 'dim_width', 'dim_height']
    importances = final_model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"Feature: {name}, Importance: {importance:.4f}")

    return final_model, scaler, train_mse, train_r2, valid_mse, valid_r2

# Step 5: Detect Flaws
def detect_flaws(df_valid, model, scaler):
    X_valid = df_valid[['dim_length', 'dim_width', 'dim_height']].values
    X_valid_scaled = scaler.transform(X_valid)
    df_valid['ideal_box_volume'] = model.predict(X_valid_scaled)
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws
    df_valid['design_flaw'] = df_valid['product_size'] > 2 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.5 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    # Outlier analysis
    print("\nValidation Set Outlier Analysis:")
    Q1 = df_valid['box_volume'].quantile(0.25)
    Q3 = df_valid['box_volume'].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_valid[df_valid['box_volume'] > Q3 + 1.5 * IQR]
    if not outliers.empty:
        print(f"Found {len(outliers)} potential outliers in validation set 'box_volume':")
        print(outliers[['ENTITY_NAME', 'box_volume', 'product_size']].to_string(index=False))
    else:
        print("No outliers detected in validation set 'box_volume'.")

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.7, test_size=0.3, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for model
    features = ['dim_length', 'dim_width', 'dim_height']
    X_train = df_train[features].values
    y_train = df_train['box_volume']
    X_valid = df_valid[features].values
    y_valid = df_valid['box_volume']

    # Train model
    model, scaler, train_mse, train_r2, valid_mse, valid_r2 = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, scaler)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_output_v5.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 0 rows with NaN in key features. Remaining: 113
Training set size: 79 records, Validation set size: 34 records

Training ideal packaging model with XGBoost...
Best parameters from GridSearchCV: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.7}
[0]	validation_0-rmse:719.70322
[1]	validation_0-rmse:707.43252
[2]	validation_0-rmse:686.23796
[3]	validation_0-rmse:670.87738
[4]	validation_0-rmse:655.49900
[5]	validation_0-rmse:641.92388
[6]	validation_0-rmse:623.65725
[7]	validation_0-rmse:611.83847
[8]	validation_0-rmse:601.47821
[9]	validation_0-rmse:594.26910
[10]	validation_0-rmse:585.17352
[11]	validation_0-rmse:578.66095
[12]	validation_0-rmse:569.59035
[13]	validation_0-rmse:561.07561
[14]	validation_0-rmse:547.72931
[15]	validation_0-rmse:542.98100
[16]	validation_0-rmse:531.03565


In [9]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn xgboost")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS with brand-specific medians
    def impute_missing(group):
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]
        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not (isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x)) else x
        )
        return group

    df = df.groupby('BRAND').apply(impute_missing, include_groups=False).reset_index()

    # Extract individual dimensions
    df['dim_length'] = df['dimensions'].apply(lambda x: x[0])
    df['dim_width'] = df['dimensions'].apply(lambda x: x[1])
    df['dim_height'] = df['dimensions'].apply(lambda x: x[2])

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Feature engineering: interaction term
    df['dim_interaction'] = df['dim_length'] * df['dim_width'] * df['dim_height']

    # Filter outliers in product_size (2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in key features
    original_len = len(df)
    df = df.dropna(subset=['dim_length', 'dim_width', 'dim_height', 'PRICE', 'product_size', 'dim_interaction'])
    print(f"Dropped {original_len - len(df)} rows with NaN in key features. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid):
    print("\nTraining ideal packaging model with XGBoost...")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Define XGBoost model for GridSearchCV
    xgb = XGBRegressor(random_state=42)
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.01],
        'subsample': [0.7, 0.9],
        'reg_lambda': [10, 20],
        'reg_alpha': [1, 5]
    }
    grid_search = GridSearchCV(
        xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
    )
    grid_search.fit(X_train_scaled, y_train)

    # Get best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters from GridSearchCV: {best_params}")

    # Train final model with early stopping
    final_model = XGBRegressor(
        random_state=42,
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        reg_lambda=best_params['reg_lambda'],
        reg_alpha=best_params['reg_alpha'],
        early_stopping_rounds=10
    )
    final_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_valid_scaled, y_valid)],
        verbose=True
    )

    # Evaluate on training set
    y_pred_train = final_model.predict(X_train_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}")

    # Evaluate on validation set
    y_pred_valid = final_model.predict(X_valid_scaled)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.4f}, Validation R²: {valid_r2:.4f}")

    # Feature importance
    feature_names = ['dim_length', 'dim_width', 'dim_height', 'PRICE', 'dim_interaction']
    importances = final_model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"Feature: {name}, Importance: {importance:.4f}")

    return final_model, scaler, train_mse, train_r2, valid_mse, valid_r2

# Step 5: Detect Flaws
def detect_flaws(df_valid, model, scaler):
    X_valid = df_valid[['dim_length', 'dim_width', 'dim_height', 'PRICE', 'dim_interaction']].values
    X_valid_scaled = scaler.transform(X_valid)
    df_valid['ideal_box_volume'] = model.predict(X_valid_scaled)
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws with adjusted threshold
    df_valid['design_flaw'] = df_valid['product_size'] > 2 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.3 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm^3 (save {row['excess_volume']:.1f} cm^3)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    # Near-flaw analysis
    print("\nNear-Flaw Analysis (Packaging Flaw Threshold > 1.2 * ideal_box_volume):")
    near_flaws = df_valid[df_valid['box_volume'] > 1.2 * df_valid['ideal_box_volume']]
    if not near_flaws.empty:
        print(f"Found {len(near_flaws)} products near packaging flaw threshold:")
        print(near_flaws[['ENTITY_NAME', 'box_volume', 'ideal_box_volume', 'recommendation']].to_string(index=False))
    else:
        print("No products near packaging flaw threshold.")

    # Outlier analysis
    print("\nValidation Set Outlier Analysis:")
    Q1 = df_valid['box_volume'].quantile(0.25)
    Q3 = df_valid['box_volume'].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_valid[df_valid['box_volume'] > Q3 + 1.5 * IQR]
    if not outliers.empty:
        print(f"Found {len(outliers)} potential outliers in validation set 'box_volume':")
        print(outliers[['ENTITY_NAME', 'box_volume', 'product_size']].to_string(index=False))
    else:
        print("No outliers detected in validation set 'box_volume'.")

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.7, test_size=0.3, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for model
    features = ['dim_length', 'dim_width', 'dim_height', 'PRICE', 'dim_interaction']
    X_train = df_train[features].values
    y_train = df_train['box_volume']
    X_valid = df_valid[features].values
    y_valid = df_valid['box_volume']

    # Train model
    model, scaler, train_mse, train_r2, valid_mse, valid_r2 = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, scaler)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_output_v6.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 0 rows with NaN in key features. Remaining: 113
Training set size: 79 records, Validation set size: 34 records

Training ideal packaging model with XGBoost...
Best parameters from GridSearchCV: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 1, 'reg_lambda': 10, 'subsample': 0.9}
[0]	validation_0-rmse:731.42848
[1]	validation_0-rmse:726.60949
[2]	validation_0-rmse:722.07058
[3]	validation_0-rmse:717.22366
[4]	validation_0-rmse:712.40280
[5]	validation_0-rmse:707.70724
[6]	validation_0-rmse:703.18972
[7]	validation_0-rmse:698.53448
[8]	validation_0-rmse:693.87938
[9]	validation_0-rmse:689.30922
[10]	validation_0-rmse:684.70345
[11]	validation_0-rmse:680.17712
[12]	validation_0-rmse:675.73152
[13]	validation_0-rmse:671.31035
[14]	validation_0-rmse:666.85178
[15]	validation_0-rmse:662.26282
[16]	validation_0-rmse:657.92962

In [10]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn xgboost")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS and ENTITY_LENGTH with brand-specific medians
    def impute_missing(group):
        # Impute dimensions
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]
        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not (isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x)) else x
        )

        # Impute ENTITY_LENGTH
        valid_lengths = group['ENTITY_LENGTH'].apply(
            lambda x: x if isinstance(x, (int, float)) and not np.isnan(x) else None
        )
        valid_lengths_list = [x for x in valid_lengths if x is not None]
        median_length = np.median(valid_lengths_list) if valid_lengths_list else df['ENTITY_LENGTH'].median()

        group['ENTITY_LENGTH'] = group['ENTITY_LENGTH'].fillna(median_length)
        return group

    df = df.groupby('BRAND').apply(impute_missing, include_groups=False).reset_index()

    # Extract individual dimensions
    df['dim_length'] = df['dimensions'].apply(lambda x: x[0])
    df['dim_width'] = df['dimensions'].apply(lambda x: x[1])
    df['dim_height'] = df['dimensions'].apply(lambda x: x[2])

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in key features
    original_len = len(df)
    df = df.dropna(subset=['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in key features. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid):
    print("\nTraining ideal packaging model with XGBoost...")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Define XGBoost model for GridSearchCV
    xgb = XGBRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 4],
        'learning_rate': [0.01, 0.05],
        'subsample': [0.8, 1.0],
        'reg_lambda': [1, 5],
        'reg_alpha': [0, 1]
    }
    grid_search = GridSearchCV(
        xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
    )
    grid_search.fit(X_train_scaled, y_train)

    # Get best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters from GridSearchCV: {best_params}")

    # Train final model with early stopping
    final_model = XGBRegressor(
        random_state=42,
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        reg_lambda=best_params['reg_lambda'],
        reg_alpha=best_params['reg_alpha'],
        early_stopping_rounds=10
    )
    final_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_valid_scaled, y_valid)],
        verbose=True
    )

    # Evaluate on training set
    y_pred_train = final_model.predict(X_train_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}")

    # Evaluate on validation set
    y_pred_valid = final_model.predict(X_valid_scaled)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.4f}, Validation R²: {valid_r2:.4f}")

    # Feature importance
    feature_names = ['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH']
    importances = final_model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"Feature: {name}, Importance: {importance:.4f}")

    return final_model, scaler, train_mse, train_r2, valid_mse, valid_r2

# Step 5: Detect Flaws
def detect_flaws(df_valid, model, scaler):
    X_valid = df_valid[['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH']].values
    X_valid_scaled = scaler.transform(X_valid)
    df_valid['ideal_box_volume'] = model.predict(X_valid_scaled)
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws with adjusted thresholds
    df_valid['design_flaw'] = df_valid['product_size'] > 1.5 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.25 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    # Near-flaw analysis
    print("\nNear-Flaw Analysis (Packaging Flaw Threshold > 1.1 * ideal_box_volume, Design Flaw > 1.3 * ideal_product_size):")
    near_packaging_flaws = df_valid[df_valid['box_volume'] > 1.1 * df_valid['ideal_box_volume']]
    near_design_flaws = df_valid[df_valid['product_size'] > 1.3 * df_valid['ideal_product_size']]
    if not near_packaging_flaws.empty:
        print(f"Found {len(near_packaging_flaws)} products near packaging flaw threshold:")
        print(near_packaging_flaws[['ENTITY_NAME', 'box_volume', 'ideal_box_volume', 'recommendation']].to_string(index=False))
    else:
        print("No products near packaging flaw threshold.")
    if not near_design_flaws.empty:
        print(f"Found {len(near_design_flaws)} products near design flaw threshold:")
        print(near_design_flaws[['ENTITY_NAME', 'product_size', 'ideal_product_size', 'recommendation']].to_string(index=False))
    else:
        print("No products near design flaw threshold.")

    # Outlier analysis
    print("\nValidation Set Outlier Analysis:")
    Q1 = df_valid['box_volume'].quantile(0.25)
    Q3 = df_valid['box_volume'].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_valid[df_valid['box_volume'] > Q3 + 1.5 * IQR]
    if not outliers.empty:
        print(f"Found {len(outliers)} potential outliers in validation set 'box_volume':")
        print(outliers[['ENTITY_NAME', 'box_volume', 'product_size']].to_string(index=False))
    else:
        print("No outliers detected in validation set 'box_volume'.")

    # Top 5 closest to flaw thresholds
    df_valid['packaging_flaw_ratio'] = df_valid['box_volume'] / df_valid['ideal_box_volume']
    df_valid['design_flaw_ratio'] = df_valid['product_size'] / df_valid['ideal_product_size']
    print("\nTop 5 Products Closest to Packaging Flaw Threshold:")
    print(df_valid.nlargest(5, 'packaging_flaw_ratio')[['ENTITY_NAME', 'box_volume', 'ideal_box_volume', 'packaging_flaw_ratio', 'recommendation']].to_string(index=False))
    print("\nTop 5 Products Closest to Design Flaw Threshold:")
    print(df_valid.nlargest(5, 'design_flaw_ratio')[['ENTITY_NAME', 'product_size', 'ideal_product_size', 'design_flaw_ratio', 'recommendation']].to_string(index=False))

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.7, test_size=0.3, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for model
    features = ['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH']
    X_train = df_train[features].values
    y_train = df_train['box_volume']
    X_valid = df_valid[features].values
    y_valid = df_valid['box_volume']

    # Train model
    model, scaler, train_mse, train_r2, valid_mse, valid_r2 = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, scaler)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_output_v7.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 2 rows with NaN in key features. Remaining: 111
Training set size: 77 records, Validation set size: 34 records

Training ideal packaging model with XGBoost...
Best parameters from GridSearchCV: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.8}
[0]	validation_0-rmse:654.79059
[1]	validation_0-rmse:631.59668
[2]	validation_0-rmse:608.72113
[3]	validation_0-rmse:592.85743
[4]	validation_0-rmse:582.24928
[5]	validation_0-rmse:565.38899
[6]	validation_0-rmse:550.90567
[7]	validation_0-rmse:537.60775
[8]	validation_0-rmse:525.63691
[9]	validation_0-rmse:513.65535
[10]	validation_0-rmse:499.95905
[11]	validation_0-rmse:493.11347
[12]	validation_0-rmse:482.69153
[13]	validation_0-rmse:472.27721
[14]	validation_0-rmse:463.32477
[15]	validation_0-rmse:452.97010
[16]	validation_0-rmse:446.38316


In [11]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import ast
import os

# Step 1: Install Dependencies
def install_dependencies():
    print("Installing dependencies...")
    os.system("pip install -q pandas numpy scikit-learn xgboost")
    print("Dependencies installed.")

# Step 2: Load and Preprocess Data
def preprocess_data(file_path):
    print("Loading data...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

    # Remove duplicates
    df = df.drop_duplicates(subset=['ENTITY_NAME', 'ENTITY_LENGTH', 'DIMENSIONS', 'PRICE', 'BRAND'])
    print(f"After removing duplicates: {len(df)} rows")

    # Parse DIMENSIONS safely
    df['dimensions'] = df['DIMENSIONS'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # Impute missing DIMENSIONS and ENTITY_LENGTH with brand-specific medians
    def impute_missing(group):
        # Impute dimensions
        valid_dims = group['dimensions'].apply(
            lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
        )
        valid_dims_list = [x for x in valid_dims if x is not None]
        if valid_dims_list:
            median_dims = np.median(valid_dims_list, axis=0)
        else:
            all_valid_dims = df['dimensions'].apply(
                lambda x: x if isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x) else None
            )
            median_dims = np.median([x for x in all_valid_dims if x is not None], axis=0)

        group['dimensions'] = group['dimensions'].apply(
            lambda x: tuple(median_dims) if x is None or not (isinstance(x, tuple) and len(x) == 3 and all(isinstance(v, (int, float)) for v in x)) else x
        )

        # Impute ENTITY_LENGTH
        valid_lengths = group['ENTITY_LENGTH'].apply(
            lambda x: x if isinstance(x, (int, float)) and not np.isnan(x) else None
        )
        valid_lengths_list = [x for x in valid_lengths if x is not None]
        median_length = np.median(valid_lengths_list) if valid_lengths_list else df['ENTITY_LENGTH'].median()

        group['ENTITY_LENGTH'] = group['ENTITY_LENGTH'].fillna(median_length)
        return group

    df = df.groupby('BRAND').apply(impute_missing, include_groups=False).reset_index()

    # Extract individual dimensions
    df['dim_length'] = df['dimensions'].apply(lambda x: x[0])
    df['dim_width'] = df['dimensions'].apply(lambda x: x[1])
    df['dim_height'] = df['dimensions'].apply(lambda x: x[2])

    # Compute product_size
    df['product_size'] = df['dimensions'].apply(
        lambda x: x[0] * x[1] * x[2] if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Compute box_volume (add 10% padding to each dimension)
    df['box_volume'] = df['dimensions'].apply(
        lambda x: (x[0] * 1.1) * (x[1] * 1.1) * (x[2] * 1.1) if all(isinstance(v, (int, float)) for v in x) else np.nan
    )

    # Filter outliers in product_size (2*IQR)
    Q1 = df['product_size'].quantile(0.25)
    Q3 = df['product_size'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[df['product_size'].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    print(f"After outlier removal: {len(df)} rows")

    # Drop rows with NaN in key features
    original_len = len(df)
    df = df.dropna(subset=['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH', 'product_size'])
    print(f"Dropped {original_len - len(df)} rows with NaN in key features. Remaining: {len(df)}")

    if len(df) < 30:
        print(f"Error: Only {len(df)} valid records available. Need at least 30.")
        return None

    return df

# Step 3: Diagnose Model Performance
def diagnose_model(train_r2, valid_r2, train_mse, valid_mse):
    print("\nModel Performance Diagnosis:")
    print(f"Training R²: {train_r2:.2f}, Validation R²: {valid_r2:.2f}")
    print(f"Training MSE: {train_mse:.2f}, Validation MSE: {valid_mse:.2f}")

    if train_r2 > 0.8 and (train_r2 - valid_r2) > 0.1 and valid_mse > 2 * train_mse:
        print("Overfitting detected: High training performance, but poor validation performance.")
        return "overfitting"
    elif train_r2 < 0.6 and valid_r2 < 0.6:
        print("Underfitting detected: Poor performance on both training and validation sets.")
        return "underfitting"
    elif abs(train_r2 - valid_r2) < 0.1 and train_r2 > 0.7:
        print("Good performance: Similar and high performance on both sets.")
        return "good"
    else:
        print("Moderate performance: Model may need slight tuning.")
        return "moderate"

# Step 4: Train Ideal Packaging Model
def train_ideal_packaging_model(X_train, y_train, X_valid, y_valid):
    print("\nTraining ideal packaging model with XGBoost...")

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Define XGBoost model for GridSearchCV
    xgb = XGBRegressor(random_state=42, early_stopping_rounds=10)
    param_grid = {
        'max_depth': [2, 3],
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05],
        'subsample': [0.7, 0.8, 0.9],
        'reg_lambda': [1, 5, 10],
        'reg_alpha': [0, 1, 5],
        'colsample_bytree': [0.7, 0.9]
    }
    grid_search = GridSearchCV(
        xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
    )
    grid_search.fit(
        X_train_scaled, y_train,
        eval_set=[(X_valid_scaled, y_valid)],
        verbose=False
    )

    # Get best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters from GridSearchCV: {best_params}")

    # Train final model with best parameters
    final_model = XGBRegressor(
        random_state=42,
        max_depth=best_params['max_depth'],
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        reg_lambda=best_params['reg_lambda'],
        reg_alpha=best_params['reg_alpha'],
        colsample_bytree=best_params['colsample_bytree'],
        early_stopping_rounds=10
    )
    final_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_valid_scaled, y_valid)],
        verbose=True
    )

    # Evaluate on training set
    y_pred_train = final_model.predict(X_train_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    print(f"Ideal Packaging Model - Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}")

    # Evaluate on validation set
    y_pred_valid = final_model.predict(X_valid_scaled)
    valid_mse = mean_squared_error(y_valid, y_pred_valid)
    valid_r2 = r2_score(y_valid, y_pred_valid)
    print(f"Ideal Packaging Model - Validation MSE: {valid_mse:.4f}, Validation R²: {valid_r2:.4f}")

    # Feature importance
    feature_names = ['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH']
    importances = final_model.feature_importances_
    for name, importance in zip(feature_names, importances):
        print(f"Feature: {name}, Importance: {importance:.4f}")

    return final_model, scaler, train_mse, train_r2, valid_mse, valid_r2

# Step 5: Detect Flaws
def detect_flaws(df_valid, model, scaler):
    X_valid = df_valid[['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH']].values
    X_valid_scaled = scaler.transform(X_valid)
    df_valid['ideal_box_volume'] = np.maximum(model.predict(X_valid_scaled), 0)  # Clip to avoid negative predictions
    df_valid['ideal_product_size'] = df_valid['ideal_box_volume'] * 0.9

    # Detect flaws with adjusted thresholds
    df_valid['design_flaw'] = df_valid['product_size'] > 1.5 * df_valid['ideal_product_size']
    df_valid['packaging_flaw'] = df_valid['box_volume'] > 1.25 * df_valid['ideal_box_volume']

    # Excess volume calculation
    df_valid['excess_volume'] = df_valid.apply(
        lambda row: row['box_volume'] - row['ideal_box_volume'] if row['packaging_flaw'] else 0,
        axis=1
    )

    # Recommendation
    df_valid['recommendation'] = df_valid.apply(
        lambda row: (
            f"Reduce box to {row['ideal_box_volume']:.1f} cm³ (save {row['excess_volume']:.1f} cm³)"
            if row['packaging_flaw']
            else "Redesign bottle to reduce volume" if row['design_flaw']
            else "No flaw detected"
        ),
        axis=1
    )

    # Near-flaw analysis
    print("\nNear-Flaw Analysis (Packaging Flaw Threshold > 1.1 * ideal_box_volume, Design Flaw > 1.3 * ideal_product_size):")
    near_packaging_flaws = df_valid[df_valid['box_volume'] > 1.1 * df_valid['ideal_box_volume']]
    near_design_flaws = df_valid[df_valid['product_size'] > 1.3 * df_valid['ideal_product_size']]
    if not near_packaging_flaws.empty:
        print(f"Found {len(near_packaging_flaws)} products near packaging flaw threshold:")
        print(near_packaging_flaws[['ENTITY_NAME', 'box_volume', 'ideal_box_volume', 'recommendation']].to_string(index=False))
    else:
        print("No products near packaging flaw threshold.")
    if not near_design_flaws.empty:
        print(f"Found {len(near_design_flaws)} products near design flaw threshold:")
        print(near_design_flaws[['ENTITY_NAME', 'product_size', 'ideal_product_size', 'recommendation']].to_string(index=False))
    else:
        print("No products near design flaw threshold.")

    # Outlier analysis
    print("\nValidation Set Outlier Analysis:")
    Q1 = df_valid['box_volume'].quantile(0.25)
    Q3 = df_valid['box_volume'].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_valid[df_valid['box_volume'] > Q3 + 1.5 * IQR]
    if not outliers.empty:
        print(f"Found {len(outliers)} potential outliers in validation set 'box_volume':")
        print(outliers[['ENTITY_NAME', 'box_volume', 'product_size']].to_string(index=False))
    else:
        print("No outliers detected in validation set 'box_volume'.")

    # Top 5 closest to flaw thresholds
    df_valid['packaging_flaw_ratio'] = df_valid['box_volume'] / df_valid['ideal_box_volume']
    df_valid['design_flaw_ratio'] = df_valid['product_size'] / df_valid['ideal_product_size']
    print("\nTop 5 Products Closest to Packaging Flaw Threshold:")
    print(df_valid.nlargest(5, 'packaging_flaw_ratio')[['ENTITY_NAME', 'box_volume', 'ideal_box_volume', 'packaging_flaw_ratio', 'recommendation']].to_string(index=False))
    print("\nTop 5 Products Closest to Design Flaw Threshold:")
    print(df_valid.nlargest(5, 'design_flaw_ratio')[['ENTITY_NAME', 'product_size', 'ideal_product_size', 'design_flaw_ratio', 'recommendation']].to_string(index=False))

    return df_valid

# Step 6: Main Execution
def main():
    install_dependencies()

    file_path = "amazon_shampoo_products.csv"
    df = preprocess_data(file_path)

    if df is None:
        return

    # Split data
    df_train, df_valid = train_test_split(df, train_size=0.7, test_size=0.3, random_state=42)
    print(f"Training set size: {len(df_train)} records, Validation set size: {len(df_valid)} records")

    # Prepare data for model
    features = ['dim_length', 'dim_width', 'dim_height', 'PRICE', 'ENTITY_LENGTH']
    X_train = df_train[features].values
    y_train = df_train['box_volume']
    X_valid = df_valid[features].values
    y_valid = df_valid['box_volume']

    # Train model
    model, scaler, train_mse, train_r2, valid_mse, valid_r2 = train_ideal_packaging_model(X_train, y_train, X_valid, y_valid)

    # Diagnose model
    diagnose_result = diagnose_model(train_r2, valid_r2, train_mse, valid_mse)

    # Detect flaws
    df_valid = detect_flaws(df_valid, model, scaler)

    # Save results
    output_df = df_valid[[
        'ENTITY_ID', 'ENTITY_NAME', 'ENTITY_LENGTH', 'product_size', 'box_volume',
        'ideal_product_size', 'ideal_box_volume', 'design_flaw', 'packaging_flaw',
        'excess_volume', 'recommendation'
    ]]
    output_file = "packaging_output_v8.csv"
    output_df.to_csv(output_file, index=False)
    print(f"\nResults saved to '{output_file}'.")

    # Summary
    design_flaws = df_valid['design_flaw'].sum()
    packaging_flaws = df_valid['packaging_flaw'].sum()
    print(f"Detected {design_flaws} products with design flaws.")
    print(f"Detected {packaging_flaws} products with packaging flaws.")

if __name__ == "__main__":
    main()

Installing dependencies...
Dependencies installed.
Loading data...
After removing duplicates: 140 rows
After outlier removal: 113 rows
Dropped 2 rows with NaN in key features. Remaining: 111
Training set size: 77 records, Validation set size: 34 records

Training ideal packaging model with XGBoost...
Best parameters from GridSearchCV: {'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.7}
[0]	validation_0-rmse:663.71626
[1]	validation_0-rmse:650.76857
[2]	validation_0-rmse:625.53973
[3]	validation_0-rmse:613.44421
[4]	validation_0-rmse:592.42930
[5]	validation_0-rmse:581.06331
[6]	validation_0-rmse:570.81988
[7]	validation_0-rmse:562.33700
[8]	validation_0-rmse:557.34860
[9]	validation_0-rmse:546.47565
[10]	validation_0-rmse:541.28576
[11]	validation_0-rmse:536.21744
[12]	validation_0-rmse:530.79215
[13]	validation_0-rmse:516.14253
[14]	validation_0-rmse:513.16012
[15]	validation_0-rmse:508.43033
[16]	val